# Introduction to uproot

The interface is minimal: open a file with `uproot.open` and extract objects with a dictionary-like interface. Let's open a NanoAOD file.

In [1]:
import uproot
tree = uproot.open("~/storage/data/nano-TTLHE-2017-09-04-lz4.root")["Events"]
tree

<TTree 'Events' len=55432 at 0x736538ed2b90>

Incidentally, the one-liner above is not possible with PyROOT because ROOT's and Python's notions of object ownership conflict.

In [ ]:
import ROOT
tree2 = ROOT.TFile("/home/pivarski/storage/data/nano-TTLHE-2017-09-04-lz4.root").Get("Events")
tree2

As with most Python modules, all of the class members and methods that don't start with underscore are public.

In [ ]:
print(", ".join(x for x in dir(tree) if not x.startswith("_")))

In [ ]:
help(tree.array)

In [ ]:
tree.branchnames

In [ ]:
tree.array("Electron_pt")

In [ ]:
tree.arrays(["Electron_pt", "Electron_eta", "Electron_phi"])

The branch/dtype argument of methods like `arrays` can take a function from `TBranch` to `dtype/None` as an argument, providing a flexible way to select branches and possibly change their Numpy `dtype` on the fly.

In [ ]:
tree.arrays(lambda branch: branch.dtype if branch.name.startswith("Electron_") else None)

For instance, we can change all numbers from the "big endian" format ROOT stores them in to your machine's native byte order.

In [ ]:
tree.arrays(lambda branch: branch.dtype if branch.name.startswith("Electron_") else None)

If the arrays are too large to read all at once, you can iterate over them.

In [3]:
import numpy
for pt, eta, phi in tree.iterator(1000, ["Electron_pt", "Electron_eta", "Electron_phi"], outputtype=tuple):
    print("px = {}".format(pt*numpy.cosh(eta)*numpy.sin(phi)))

px = [-22.05850983   2.0827117  -11.79761791 ...,  31.39526558  10.02485085
  -5.15552425]
px = [  57.21009064  -21.60737228   51.51699066 ..., -232.88720703   83.01596069
   -8.48415852]
px = [-16.30791855 -11.02349567  -9.59561825 ...,   1.86529851   7.97578096
  93.77963257]
px = [ 31.80595207 -27.0686264   17.70536232 ...,   9.13830853  -8.50615883
 -15.53428745]
px = [-15.63441658  66.32009125 -23.5351181  ...,  37.11478424 -82.37744141
  -5.42560482]
px = [ -1.71087015   9.87014294  34.29794312 ..., -20.05436897  54.53624344
 -17.53446388]
px = [ 45.2231369    2.15468979  17.36893845 ...,   9.81973648 -29.31288338
  -8.11543274]
px = [-30.55264282   3.06022358 -25.02843285 ...,  15.1167841  -34.62071991
  -5.77784777]
px = [ -1.69409871  -5.72944546  -6.32986736 ..., -40.91424561  11.66949177
  14.31631088]
px = [ 18.41147804  10.69971561 -75.66543579 ...,  55.98440933  -4.252491
  33.29589081]
px = [ 10.72761917 -26.83973503 -11.02705956 ..., -32.10058594 -11.14105988
 -16.03596

Or over a collection of files (like TChain).

In [4]:
for pt, eta, phi in uproot.iterator(1000, "~/storage/data/nano-TTLHE-2017-09-04-*.root", "Events", ["Electron_pt", "Electron_eta", "Electron_phi"], outputtype=tuple):
    print("px = {}".format(pt*numpy.cosh(eta)*numpy.sin(phi)))

px = [-22.05850983   2.0827117  -11.79761791 ...,  31.39526558  10.02485085
  -5.15552425]
px = [  57.21009064  -21.60737228   51.51699066 ..., -232.88720703   83.01596069
   -8.48415852]
px = [-16.30791855 -11.02349567  -9.59561825 ...,   1.86529851   7.97578096
  93.77963257]
px = [ 31.80595207 -27.0686264   17.70536232 ...,   9.13830853  -8.50615883
 -15.53428745]
px = [-15.63441658  66.32009125 -23.5351181  ...,  37.11478424 -82.37744141
  -5.42560482]
px = [ -1.71087015   9.87014294  34.29794312 ..., -20.05436897  54.53624344
 -17.53446388]
px = [ 45.2231369    2.15468979  17.36893845 ...,   9.81973648 -29.31288338
  -8.11543274]
px = [-30.55264282   3.06022358 -25.02843285 ...,  15.1167841  -34.62071991
  -5.77784777]
px = [ -1.69409871  -5.72944546  -6.32986736 ..., -40.91424561  11.66949177
  14.31631088]
px = [ 18.41147804  10.69971561 -75.66543579 ...,  55.98440933  -4.252491
  33.29589081]
px = [ 10.72761917 -26.83973503 -11.02705956 ..., -32.10058594 -11.14105988
 -16.03596

px = [ -3.93797646e+01  -5.46227515e-01   3.27955794e+00 ...,   2.72346139e-01
   6.06136990e+00  -5.74205627e+02]
px = [ -5.79825592   6.32270336   6.36440182 ...,  21.06732368   5.91692162
 -27.29461098]
px = [ -5.21661472  20.11013985  34.26156235 ..., -35.20895386  32.28709412
  14.42957592]
px = [-43.39455032 -33.0429039   12.04433727 ...,   3.25852895 -26.25276184
  16.41854668]
px = [ 23.40258408  37.5812912   20.22359848 ...,  15.93418026  19.07468796
 -40.36912918]
px = [  9.40824509   5.31695366  19.40161514 ..., -17.33235168 -24.25399971
  33.5326004 ]
px = [-12.05172729  75.03172302 -34.71931076 ...,  35.78233337 -12.72197247
   8.45560551]
px = [  3.27930832 -67.28022766 -22.99330139 ...,  14.94455719  11.15563297
  37.79850388]
px = [  2.14220309  18.06168938 -13.75863075 ...,   5.92840624 -55.38665771
  31.06835365]
px = [-39.56405258 -58.99796295  -6.06731701 ..., -13.87748623  13.28912735
 -22.45241928]
px = [ 64.20173645  21.165411    69.2530899  ...,  18.75812912  59